# Connecting to Postgres Lab

### Introduction

In this lesson, we'll setup an RDS instance and use it to serve as the database for our foursquare flask api.  Let's get started.

### Setting it up

* We should begin by going to the AWS console, finding the RDS service, and creating a new database.  
* Then select the postgres database, and the free tier.
* From here, it's time to add in the settings for the database.  Choose an instance identifier of `foursquare-flask-api`.  And set a master username and password.  

> **Note**: Make sure to write down these values as we'll need them to connect to our database later.

* Then, under connectivity make sure that there is public access to the database.  

> This will allow us to access the database with just the ip address, username, and password.

* After checking that the configuration is correct, create the database.

### Checking our Work

Go to the dashboard and click on the foursquare-flask-api.  While it is still just creating, we will be able to see the settings of the database instance.

<img src="./foursquare_flask_api.png" width="100%">

If we click on it, we'll see the following.

<img src="./foursquare_api.png" width="100%">

### Connecting to our instance

Now let's connect to our instance and create a database called `foursquare_production`.

Once connected, listing the databases and we should see the following.

<img src='./listing_databases.png' width="80%">

From there, we can run migrations to create the production tables.  

> We can use the same `psql` command we normally do to run migrations, just this time specify a `--host` parameter along with the file, and the database with `-d`.  The host parameter will be the public ip address for the database.

If you run the following from the shell, filling in the value for the host, you can check that the tables were created.

`psql -U postgres -d foursquare_production --host=specify_host -c "\dt"`

<img src="./view_relations.png" width="60%">

### Connecting our Application

Ok, now let's connect our flask application to our production database.  We'll do this in two steps.  First, we'll first make the change locally and check that it's working.  And then we'll apply the change to our Flask API running on the EC2 instance.

1. Making the Change Locally

To connect our local flask application to the production database.  We need to change our environmental variables.  Remember that our environmental variables are stored in the `.env` file.  We can change those values locally, and run our application locally to that we are connected to the RDS instance.

> Run the application using a port of `0.0.0.0`.

> If you visit the venues resource, we should see the following.

<img src="./venues.png" width="80%">

This is great, as we are now connected to our database on AWS.

2. Connecting on AWS

A. Get the Flask Code Running

* **Planning**
Now with connecting our database to our code on AWS, we mean our foursquare-flask-api.  Now if we do not have the `foursquare-flask-api` running on AWS, we can move the code over to an appropriate instance, and then try to get it running without a database, and later with a database.


a. Deploy to EC2 without a database

Without a connection to the database, we can boot up the code to make sure it's working properly, but will need to first make a a couple of changes.  

In the `backend/api/src/__init__.py`, we can find the following lines:

```python
from flask import Flask
import simplejson as json
from flask import request

import api.src.models as models
import api.src.db as db
```

The bottom two lines, beginning with api.src both rely on the database. Delete those two lines.

```python
import api.src.models as models
import api.src.db as db
```

* We will also have to change the `run.py` file so that we are running on the correct host and port.  
* Ok, then `scp` the code over to the running EC2 instance, or create a new EC2 instance if one is not available.

* Connect to the instance, create a new tmux session, and boot up the flask app with a call to `sudo python3 run.py`.  Detach from the tmux session, and confirm that we our application works by making a request (via HTTP not HTTPS in the browser).

2. Connect to the correct database

* Ok, now perhaps the first change we should make is to add back in the lines of code we deleted previously, so that our `backend/api/src/__init__.py` file looks like the following:

```python
from flask import Flask
import simplejson as json
from flask import request

import api.src.models as models
import api.src.db as db
```

> Either make this change by using vim or make the change locally and copy the code over to the backend.

* Next it's time to make the change to the `.env` file that we made locally onto our AWS EC2 instance.  Let's find the public ip address, remove the existing `.env` file and use `scp` to replace it with the updated `.env` file.  

Then `ssh` into the AWS machine and confirm that the `.env` file has been properly updated.

>Run `cat backend/.env` to check that the changes have been made.

If the `.env` file has been properly updated, then attach to the session that is running the flask application.

`tmux attach-session -t flask_lab`

And then shutdown and rerun the flask application with host `0.0.0.0` and port `80`.  Our flask application should now be able to connect to the database without breaking (there just won't be any data in there).

<img src="./venues_from_ec2.png" width="80%">

### Adding some data

Let's pull down some venues into our production database by using our application code to connect to the foursquare API and filling our database with venue records.  

> We can do this by again sshing into our EC2 instance and this time running:

> `sudo python3 manage.py build_venues "40.7,-74" "tacos"`

This uses the command defined in the manage.py file.  Refreshing the venues url from our api, we should see the following.

<img src="./production_venues_filled.png" width="80%">

### Summary

In this lesson, we connected our application to our production database.  The key steps were to create our RDS instance while making our RDS instance publicly accessible.  Then we need to configure our RDS instance with the master username, password.  This along with the RDS ip address, allowed us to connect to our instance create our production database and run our migration files.

We then change our username, password, and host address in our `.env` file to our flask application reference the correct database in production.  We updated the code on our EC2 instance, and re-ran our flask application with in tmux.  Then after getting this working, we used our cli command to populate our database.  